# 统计计算

## 分布

一组值及其对应的概率，例如骰子的1~6，及其对应的1/6（骰子质地均匀）；

## 曲奇饼问题 - Pmf解决

问题：有两个碗，碗1中有30个香草曲奇，10个巧克力曲奇，碗2中各有20个，问随便拿一个曲奇，从碗1取到香草的概率是多少？

In [15]:
# 可能1：碗1，可能2：碗2
from code.thinkbayes import Pmf
pmf = Pmf() # 创建概率质量函数对象
pmf.Set('Bowl 1',0.5) # 创建可能1，先验概率为0.5
pmf.Set('Bowl 2',0.5) # 创建可能2，先验概率为0.5
pmf.Mult('Bowl 1',0.75) # 可能1乘以似然度0.75
pmf.Mult('Bowl 2',0.5) # 可能1乘以似然度0.5
pmf.Normalize()
pmf.Prob('Bowl 1')

0.6000000000000001

## M&M问题 - Pmf解决

问题：有两袋M&M豆，94袋中30%褐色，20%黄色，20%红色，10%绿色，10%橙色，10%黄褐色，96袋中24%蓝色，20%绿色，16%橙色，14%黄色，13%红色，13%褐色，从两袋中各取一个豆，一个是黄色，一个是绿色，问黄色豆来自94年的袋子的概率是多少？

In [16]:
# 可能1：黄色来自94，绿色来自96，可能2：黄色来自96，绿色来自94
from code.thinkbayes import Pmf
pmf = Pmf() # 创建概率质量函数对象
pmf.Set('94,96',0.5) # 创建可能1，先验概率为0.5
pmf.Set('96,94',0.5) # 创建可能2，先验概率为0.5
pmf.Mult('94,96',0.2*0.2) # 可能1乘以似然度
pmf.Mult('96,94',0.14*0.1) # 可能1乘以似然度
pmf.Normalize()
pmf.Prob('94,96')

0.7407407407407408

## Monty Hall问题 - Pmf解决

蒙梯大厅问题：三扇门，其中一扇后有奖品，你随便挑选一扇，然后会展示剩下两扇中一扇没有奖品的门，然后你再做出选择，是保持原来的选择，还是从新选择另一扇没被打开的门；

In [17]:
# 选手选A，Monty打开门B且奖品不在B
# 可能1：奖品在A，可能2：奖品在B，可能3：奖品在C
from code.thinkbayes import Pmf
pmf = Pmf() # 创建概率质量函数对象
pmf.Set('A',1./3) # 创建可能1，先验概率为1/3
pmf.Set('B',1./3) # 创建可能2，先验概率为1/3
pmf.Set('C',1./3) # 创建可能3，先验概率为1/3
pmf.Mult('A',1./2) # 可能1乘以似然度
pmf.Mult('B',0) # 可能1乘以似然度
pmf.Mult('C',1) # 可能1乘以似然度
pmf.Normalize()
pmf.Prob('C') # 即选手改变自己的选择(由于B是被Monty打开的，因此选手只能改为C)的话选中概率

0.6666666666666666

## 贝叶斯框架 -- 曲奇饼问题

该框架的好处：
1. 很多问题可以改改就通用了，后面会看到；
2. 能够推广到拿多个饼，各个口味的概率；

### 构建Cookie类，基于Pmf类

In [18]:
class Cookie(Pmf):
    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
    }
    
    def __init__(self, hypos):
        '''
        Cookie类构造函数
        
        Args:
            hypos -- 全部假设
        '''
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        '''
        根据传入data（此处是口味）求似然度
        
        Args:
            data -- 传入的信息，此处是口味
            hypo -- 某一种假设
            
        Returns:
            like -- 更新后的概率
        '''
        mix = self.mixes[hypo]
        like = mix[data]
        return like
        
    def Update(self, data):
        '''
        修正相应假设的概率
        
        Args:
            data -- 用于修正响应假设概率的信息，此处就是饼干口味
        '''
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def PrintPredict(self):
        '''
        打印各个假设及其对应的概率
        '''
        for hypo, prob in self.Items():
            print hypo + ':' + str(prob)
        

### 测试Cookie类

#### 原始概率 -- 即先验概率，构造中定义了都是1

In [19]:
cookie = Cookie(['Bowl 1', 'Bowl 2'])
cookie.PrintPredict()

Bowl 2:0.5
Bowl 1:0.5


#### 更新一个香草信息

In [20]:
cookie.Update('vanilla')
cookie.PrintPredict()

Bowl 2:0.4
Bowl 1:0.6


#### 更新一个巧克力

In [21]:
cookie.Update('chocolate')
cookie.PrintPredict()

Bowl 2:0.571428571429
Bowl 1:0.428571428571


## 贝叶斯框架 -- Monty Hall问题

### 构建MontyHall类，继承自Pmf类

In [26]:
class MontyHall(Pmf):
    '''
    假设用户最开始选的是门A
    信息是主持人打开门B
    '''
    
    mixes = {
        'Gate A has price':dict(A=0, B=0.5, C=0.5),
        'Gate B has price':dict(A=0, B=0, C=1.),
        'Gate C has price':dict(A=0, B=1., C=0),
    }
    
    def __init__(self, hypos):
        '''
        MontyHall类构造函数
        
        Args:
            hypos -- 全部假设
        '''
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        '''
        根据传入data（此处是主持人去掉某个门）求似然度
        
        Args:
            data -- 传入的信息，此处是去掉的某扇门
            hypo -- 某一种假设
            
        Returns:
            like -- 更新后的概率
        '''
        mix = self.mixes[hypo]
        like = mix[data]
        return like
        
    def Update(self, data):
        '''
        修正相应假设的概率
        
        Args:
            data -- 用于修正响应假设概率的信息，此处就是某扇门
        '''
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def PrintPredict(self):
        '''
        打印各个假设及其对应的概率
        '''
        for hypo, prob in self.Items():
            print hypo + ':' + str(prob)
        

### 测试MontyHall类

#### 原始概率

In [27]:
montyhall = MontyHall(['Gate A has price', 'Gate B has price', 'Gate C has price'])
montyhall.PrintPredict()

Gate B has price:0.333333333333
Gate C has price:0.333333333333
Gate A has price:0.333333333333


#### 加入主持人信息

In [28]:
montyhall.Update('B')
montyhall.PrintPredict()

Gate B has price:0.0
Gate C has price:0.666666666667
Gate A has price:0.333333333333


In [29]:
montyhall.Update('C')
montyhall.PrintPredict()

Gate B has price:0.0
Gate C has price:0.0
Gate A has price:1.0


#### 加入主持人去掉一扇门后的概率